In [1]:
import os
import pandas as pd
from os.path import dirname

root_dir = dirname(os.getcwd())
os.chdir(root_dir)

In [2]:
location = 'sanjuan'
data_split = 'calibration'

In [3]:
# Bello
# "latitude": 6.3373,
# "longitude": -75.558,

# Iquitos
# "latitude": -3.7491,
# "longitude": -73.2538,

# San Juan
# "latitude": 18.4663,
# "longitude": -66.1057,

In [4]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 18.4663,
	"longitude": -66.1057,
	"start_date": "2000-12-18",
	"end_date": "2001-11-18",
	"daily": ["temperature_2m_mean", "rain_sum"],
	"timezone": "auto",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_rain_sum = daily.Variables(1).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["rain_sum"] = daily_rain_sum

daily_dataframe = pd.DataFrame(data = daily_data)

Coordinates: 18.453426361083984°N -66.13504028320312°E
Elevation: 15.0 m asl
Timezone: b'America/Puerto_Rico'b'GMT-4'
Timezone difference to GMT+0: -14400s


In [5]:
temp_df = daily_dataframe.copy()
temp_df['date'] = pd.to_datetime(temp_df['date'])

temp_df = temp_df[['date', 'temperature_2m_mean', 'rain_sum']]

for col in ['rain_sum', 'temperature_2m_mean']:
    temp_df[col] = pd.to_numeric(temp_df[col], errors='coerce')

weekly_df = temp_df.groupby(pd.Grouper(key='date', freq='W')).mean().reset_index()
weekly_df.to_csv(f'data/{data_split}/{location}/weather_weekly.csv', index=False)